# PL11 .- QUANTUM NEURAL NETWORKS

En este notebook, vamos a mostrar cómo se pueden construir redes neuronales cuánticas integrando *PennyLane* con *Keras*.

Comenzamos importando las librerías necesarias y fijando las semillas de los generadores aleatorios.


In [ ]:
# !pip install pennylane
# !pip install tensorflow==2.15
# !pip install silence_tensorflow

import matplotlib
from matplotlib import pyplot as plt
import numpy as np
import tensorflow as tf
import pennylane as qml
import random
import os

os.environ['PYTHONHASHSEED']='0'

rs = 12345  # Semilla aleatoria

random.seed(rs)
np.random.seed(rs)
tf.random.set_seed(rs)

from silence_tensorflow import silence_tensorflow
silence_tensorflow()

Creamos un conjunto de datos sencillo y los dividimos en entrenamiento y test.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_blobs

X, y = make_blobs(n_samples=200, centers = [[0,2],[2,0]], random_state = rs)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify = y, random_state=rs)
plt.scatter(X[:, 0], X[:, 1], c = y, cmap=matplotlib.colors.ListedColormap(["red","green"]));

Como vamos a usar *angle embedding*, escalamos los datos al intervalo $[0,\pi]$.

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(0,np.pi))
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

## REDES NEURONALES CUÁNTICAS

Definimos un circuito que entrenaremos para crear nuestra red neuronal cuántica. Como *feature map* usamos *angle embedding* (https://docs.pennylane.ai/en/stable/code/api/pennylane.AngleEmbedding.html) y como forma variacional utilizamos *BasicEntanglerLayers* (https://docs.pennylane.ai/en/stable/code/api/pennylane.BasicEntanglerLayers.html). Finalmente, medimos para comprobar si el primer qubit es 0 o 1.

Ya que el conjunto de datos tiene dos atributos, usamos dos qubits.  

In [ ]:
n_qubits = 2
dev = qml.device("default.qubit", wires=n_qubits)

@qml.qnode(dev)
def qnode(inputs, weights):
    qml.templates.AngleEmbedding(inputs, wires=range(n_qubits))
    qml.templates.BasicEntanglerLayers(weights, wires=range(n_qubits))
    # Probablilidad de que el primer qubit sea un 0 o un 1
    state_0 = [[1], [0]]
    y = state_0 * np.conj(state_0).T
    return [qml.expval(qml.Hermitian(y, wires=[0]))]

Para poder conectar *PennyLane* con *Keras* usamos *KerasLayer*, a la que debemos pasar un argumento *weight_shapes* con el formato de los parámetros de la forma variacional (los parámetros que se entrenarán).

Probamos primero con una sola repetición de la forma variacional (*n_layers=1*).

In [ ]:
n_layers = 1
weight_shapes = {"weights": (n_layers, n_qubits)}
qlayer = qml.qnn.KerasLayer(qnode, weight_shapes, output_dim=n_qubits)

Construímos un modelo de Keras que tiene como única capa el circuito cuántico definido arriba. Para que el entrenamiento no sea excesivamente lento, aumentamos el *learning rate* hata 0.05.

In [ ]:
model = tf.keras.models.Sequential([qlayer])
opt = tf.keras.optimizers.Adam(learning_rate=0.05)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=["accuracy"])

Entrenamos durante 100 épocas con *early stopping*.

In [ ]:
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
history = model.fit(X_train, y_train, epochs=100, validation_split=0.2, callbacks=[early_stopping_cb])

In [ ]:
import pandas as pd

pd.DataFrame(history.history).plot(figsize=(8, 5))

model.evaluate(X_test, y_test)

Los resultados son malos, pero es que el modelo sólo tiene dos parámetros.

In [ ]:
model.summary()

Probamos ahora con dos repeticiones de la forma variacional.

In [ ]:
model = tf.keras.models.Sequential([qlayer])
opt = tf.keras.optimizers.Adam(learning_rate=0.05)
model.compile(loss='binary_crossentropy', optimizer = opt, metrics=["accuracy"])

In [ ]:
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
history = model.fit(X_train, y_train, epochs=100, validation_split=0.2, callbacks=[early_stopping_cb])

In [ ]:
import pandas as pd

pd.DataFrame(history.history).plot(figsize=(8, 5));

model.evaluate(X_test, y_test)

In [ ]:
model.summary()

Probemos a cambiar el tipo de forma variacional a *StronglyEntanglingLayers* (https://docs.pennylane.ai/en/stable/code/api/pennylane.StronglyEntanglingLayers.html).

Nótese que ahora tenemos 3 parámetros por cada puerta de rotación.  

In [ ]:
@qml.qnode(dev)
def qnode(inputs, weights):
    qml.templates.AngleEmbedding(inputs, wires=range(n_qubits))
    qml.templates.StronglyEntanglingLayers(weights, wires=range(n_qubits))
    state_0 = [[1], [0]]
    y = state_0 * np.conj(state_0).T
    return [qml.expval(qml.Hermitian(y, wires=[0]))]

n_layers = 1
weight_shapes = {"weights": (n_layers, n_qubits,3)}
qlayer = qml.qnn.KerasLayer(qnode, weight_shapes, output_dim=n_qubits)

model = tf.keras.models.Sequential([qlayer])
opt = tf.keras.optimizers.Adam(learning_rate=0.05)
model.compile(loss='binary_crossentropy', optimizer = opt, metrics=["accuracy"])

In [ ]:
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=10,
                                                  restore_best_weights=True)
history = model.fit(X_train, y_train, epochs=100,
                    validation_split=0.2, callbacks=[early_stopping_cb])

In [ ]:
import pandas as pd

pd.DataFrame(history.history).plot(figsize=(8, 5));

model.evaluate(X_test, y_test)

In [ ]:
model.summary()

Ahora, con dos repeticiones de la forma variacional.

In [ ]:
@qml.qnode(dev)
def qnode(inputs, weights):
    qml.templates.AngleEmbedding(inputs, wires=range(n_qubits))
    qml.templates.StronglyEntanglingLayers(weights, wires=range(n_qubits))
    state_0 = [[1], [0]]
    y = state_0 * np.conj(state_0).T
    return [qml.expval(qml.Hermitian(y, wires=[0]))]

n_layers = 2
weight_shapes = {"weights": (n_layers, n_qubits,3)}
qlayer = qml.qnn.KerasLayer(qnode, weight_shapes, output_dim=n_qubits)

model = tf.keras.models.Sequential([qlayer])
opt = tf.keras.optimizers.Adam(learning_rate=0.05)
model.compile(loss='binary_crossentropy', optimizer = opt, metrics=["accuracy"])

In [ ]:
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=10,
                                                  restore_best_weights=True)
history = model.fit(X_train, y_train, epochs=100,
                    validation_split=0.2, callbacks=[early_stopping_cb])

In [ ]:
import pandas as pd

pd.DataFrame(history.history).plot(figsize=(8, 5));

model.evaluate(X_test, y_test)

In [ ]:
model.summary()

## EJERCICIO

Crear una red cuántica para un problema de clasificación binaria con 4 variables de entrada. ¿Cuántos qubits tienes que usar en la capa cuántica?

In [ ]:
@qml.qnode(dev)
def qnode(inputs, weights):
    qml.templates.AngleEmbedding(inputs, wires=range(n_qubits))
    qml.templates.StronglyEntanglingLayers(weights, wires=range(n_qubits))
    state_0 = [[1], [0]]
    y = state_0 * np.conj(state_0).T
    return [qml.expval(qml.Hermitian(y, wires=[0]))]

n_layers = 4
weight_shapes = {"weights": (n_layers, n_qubits,3)}
qlayer = qml.qnn.KerasLayer(qnode, weight_shapes, output_dim=n_qubits)

model = tf.keras.models.Sequential([qlayer])
opt = tf.keras.optimizers.Adam(learning_rate=0.05)
model.compile(loss='binary_crossentropy', optimizer = opt, metrics=["accuracy"])

In [ ]:
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=10,
                                                  restore_best_weights=True)
history = model.fit(X_train, y_train, epochs=100,
                    validation_split=0.2, callbacks=[early_stopping_cb])

In [ ]:
import pandas as pd

pd.DataFrame(history.history).plot(figsize=(8, 5));

model.evaluate(X_test, y_test)

In [ ]:
model.summary()